In [1]:
# TO FIX:
# add month-actual index
# month-actual adjusted interest rate
# might want to account for leap years somehow


from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import re

# function created for this project (should create link if I ever share this)
from validate_date import validate_date

def loan_amortization(principal, interest_rate, start_date, end_date, periodtype):
    
    """ 
        Args:
        principal (float): The principal amount of the loan.
        interest_rate (float): The annual interest rate, expressed as a percent (e.g. 5% = 5).
        start_date (str or datetime): The start date of the loan in 'YYYY-MM-DD' format or as a datetime object.
        end_date (str or datetime): The end date of the loan in 'YYYY-MM-DD' format or as a datetime object.
        periodtype (str): The type of period to use for the loan payments, which can be one of the following:
            'D' (days), 'W' (weeks), 'BW' (biweekly), 'M-30' (months 30/360), 'M-Actual' (months days_in_month/360), or 'Y' (years).

    Returns:
        pandas.DataFrame: A DataFrame containing the amortization schedule for the loan, with columns for the payment number, payment date, payment amount, interest paid, principal paid, and balance.
    """
    
    if validate_date(start_date,end_date) == False:
        raise TypeError("start_date and end_date must be a string in 'YYYY-MM-DD' format or a datetime object")
    
    # if the date is in the string format, convert it
    if not isinstance(start_date, datetime):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if not isinstance(end_date, datetime):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
                        
    # input type checking for principal and interest_rate
    if not isinstance(principal, (int, float)):
        raise TypeError("Principal amount should be numeric (int or float)")
    if not isinstance(interest_rate, (int, float)):
        raise TypeError("Interest rate should be numeric and in % (int or float)")
        
    # period-type definition 
    if periodtype == 'D':
        periods = int((end_date - start_date).days)
        days_between_payments = 1
        adjusted_rate = interest_rate / 36500
    elif periodtype == 'W':
        periods = int((end_date - start_date).days / 7)
        days_between_payments = 7
        adjusted_rate = interest_rate / 5200
    elif periodtype == 'BW':
        periods = int((end_date - start_date).days / 14)
        days_between_payments = 14
        adjusted_rate = interest_rate / 2600
    elif periodtype == 'M-30':
        periods = int((end_date - start_date).days / 30)
        days_between_payments = 30
        adjusted_rate = interest_rate / 1200
    elif periodtype == 'M-Actual':
        if start_date.month < end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1)
        elif start_date.month > end_date.month:
            periods = int((end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 1)
        else:
            periods = int((end_date.year - start_date.year) * 12)
        adjusted_rate = interest_rate / 1200  # this is wrong, should divide by 360 and multiply by days in current month
    elif periodtype == 'Y':
        periods = int(end_date.year - start_date.year)
        days_between_payments = 365
        adjusted_rate = interest_rate 
    else:
        raise TypeError("periodtype should be one of the following: 'D', 'W', 'BW', 'M-30', 'M-Actual', 'Y'")
    
    monthly_payment = principal * adjusted_rate / (1 - (1 + adjusted_rate) ** (-periods))

    # create a list of dates for each payment
    if not periodtype == 'M-Actual':
        payment_dates = [start_date + relativedelta(days=(days_between_payments * i)) for i in range(periods)]
    else:
        payment_dates = "gotta add month-actual index"

    # lists for the payment number, payment amount, interest, principal, and balance
    payment_number = list(range(1, periods + 1))
    payment_amount = monthly_payment
    interest = []
    principal_paid = []
    balance = [principal]

    # interest, principal, and balance for each payment period
    for i in range(periods):
        interest.append(round(balance[i] * adjusted_rate, 2))
        principal_paid.append(round(monthly_payment - interest[i], 2))
        balance.append(round(balance[i] - principal_paid[i], 2))

    # make the amortization-schedule dataframe
    data = {
        'Payment Number': payment_number,
        'Payment Date': payment_dates,
        'Payment Amount': payment_amount,
        'Interest Paid': interest,
        'Principal Paid': principal_paid,
        'Balance': balance[:-1]
    }
    df = pd.DataFrame(data)

    # set the index to the payment date
    df.set_index('Payment Date', inplace=True)


    # # Truncate the DataFrame to the end date
    # df = df.truncate(before=start_date, after=end_date)

    # apply formating for dollar signs and two decimals
    df.index.name = 'Date'
    df['Payment Amount'] = df['Payment Amount'].apply(lambda x: '${:,.2f}'.format(x))
    df['Interest Paid'] = df['Interest Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Principal Paid'] = df['Principal Paid'].apply(lambda x: '${:,.2f}'.format(x))
    df['Balance'] = df['Balance'].apply(lambda x: '${:,.2f}'.format(x))

    return df

In [2]:
start_date = datetime(2022, 1, 15)
end_date = datetime(2022, 5, 15)

num_days_per_month = []
while start_date <= end_date:
    days_in_month = (start_date + relativedelta(day=1, months=+1, days=-1)).day
    start_date = start_date + relativedelta(months=+1)
    num_days_per_month.append(days_in_month)

print(num_days_per_month)

[31, 28, 31, 30, 31]


In [5]:
# Test: 5-year loan, starting on January 1, 2023 and ending on December 31, 2027
schedule = loan_amortization(100000, 5, "2023-1-1", "2027-12-31", 'M-Actual')

schedule

,Payment Number,Payment Amount,Interest Paid,Principal Paid,Balance
Date,,,,,
gotta add month-actual index,1,"$1,887.12",$416.67,"$1,470.45","$100,000.00"
gotta add month-actual index,2,"$1,887.12",$410.54,"$1,476.58","$98,529.55"
gotta add month-actual index,3,"$1,887.12",$404.39,"$1,482.73","$97,052.97"
gotta add month-actual index,4,"$1,887.12",$398.21,"$1,488.91","$95,570.24"
gotta add month-actual index,5,"$1,887.12",$392.01,"$1,495.11","$94,081.33"
gotta add month-actual index,6,"$1,887.12",$385.78,"$1,501.34","$92,586.22"
gotta add month-actual index,7,"$1,887.12",$379.52,"$1,507.60","$91,084.88"
gotta add month-actual index,8,"$1,887.12",$373.24,"$1,513.88","$89,577.28"
gotta add month-actual index,9,"$1,887.12",$366.93,"$1,520.19","$88,063.40"
